<a href="https://www.kaggle.com/code/andrey36912/notebookb7111f7b82?scriptVersionId=194419135" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/technology_data.csv
/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/sports_data.csv
/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/education_data.csv
/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/entertainment_data.csv
/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/business_data.csv


In [20]:
!pip install torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.4 MB/s eta 0:00:00:00:010:01


In [21]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
import torchtext

OSError: /opt/conda/lib/python3.10/site-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
technology_df = pd.read_csv("/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/technology_data.csv")
sports_df = pd.read_csv("/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/sports_data.csv")
education_df = pd.read_csv("/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/education_data.csv")
entertainment_df = pd.read_csv("/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/entertainment_data.csv")
business_df = pd.read_csv("/kaggle/input/news-articles-classification-dataset-for-nlp-and-ml/business_data.csv")

In [ ]:
technology_df['labels'] = 'technology'
sports_df['labels'] = 'sports'
education_df['labels'] = 'education'
entertainment_df['labels'] = 'entertainment'
business_df['labels'] = 'business'

In [ ]:
df = pd.concat([technology_df, sports_df, education_df, entertainment_df, business_df], ignore_index=True)
df['text'] = df['headlines'].fillna('') + ' ' + df['description'].fillna('') + ' ' + df['content'].fillna('')
df.drop(['headlines', 'description', 'content', 'url'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, y, max_len=100):
        self.X = X
        self.y = y
        self.max_len = max_len
        
        self.label_encoder = LabelEncoder().fit(y)
        self.vocab = torchtext.vocab.Vectors("/kaggle/input/glove6b/glove.6B.50d.txt")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        label = self.label_encoder.transform([self.y.iloc[idx]])
        label = torch.tensor(label)
        
        text = self.X.iloc[idx]
        tokens = text.split()
        
        if len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
        else:
            diff = self.max_len - len(tokens)
            tokens += ['<pad>'] * diff
        
        X = self.vocab.get_vecs_by_tokens(tokens, lower_case_backup=True)
        
        return X, label[0]


dataset = MyDataset(df['text'], df['labels'])